In [16]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

def isDigitStart(word):
    # 1. The first character is a digit (other characters can be digits)
    if re.match("^[0-9]", word):
        return True

    return False


def isName(word):
    if word == None:
        return False

    # 2. The first character is uppercase
    if not re.match("^[A-Z]", word):
        return False

    #  3. All the other characters must not be uppercase
    if len(word) > 1:
        for c in word[1:]:
            if  re.match("[A-Z]", c):
                return False
    
    return True


for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)

    for word in words:
#         print >> sys.stderr, "reporter:counter:myGroup,Total_words,%d" % 1
        
        if isDigitStart(word):
            continue

        if isName(word):
            label = 'ok'
        else:
            label = 'not_ok'
            
        print "%s\t%d\t%s" % (word.lower(), 1, label)

Overwriting mapper.py


In [12]:
%%writefile reducer.py

import sys

current_key = None
word_sum_ok = 0
word_sum_not_ok = 0

for line in sys.stdin:
    try:
        key, count, label = line.strip().split('\t', 2)
        count = int(count)
    except ValueError as e:
        continue
    
    # New key
    if current_key != key:
        if current_key and word_sum_ok != 0 and float(word_sum_not_ok) / float(word_sum_ok + word_sum_not_ok) <= 0.005:
            print "%s\t%d" % (current_key, word_sum_ok)
        word_sum_ok = 0
        word_sum_not_ok = 0
        current_key = key
    
    # Same key
    if label == 'ok':
        word_sum_ok += count
    else:
        word_sum_not_ok += count

if current_key and word_sum_ok != 0 and float(word_sum_not_ok) / float(word_sum_ok + word_sum_not_ok) <= 0.005:
    print "%s\t%d" % (current_key, word_sum_ok)

Overwriting reducer.py


In [13]:
%%bash

OUT_DIR="wordcount_result"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> err.log

# Print result
hadoop fs -cat ${OUT_DIR}/part-* | sort -k2,2nr | head

# print log to stderr for grader
cat err.log >&2

american	17215
english	7794
british	6668
john	5762
french	5742
german	4622
york	4457
roman	3805
europe	3586
european	3528


sort: write failed: 'standard output': Broken pipe
sort: write error
18/01/23 06:23:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/23 06:23:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/23 06:23:58 INFO mapred.FileInputFormat: Total input files to process : 1
18/01/23 06:23:58 INFO mapreduce.JobSubmitter: number of splits:2
18/01/23 06:23:59 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1516676845530_0022
18/01/23 06:23:59 INFO impl.YarnClientImpl: Submitted application application_1516676845530_0022
18/01/23 06:23:59 INFO mapreduce.Job: The url to track the job: http://662e0c8f470a:8088/proxy/application_1516676845530_0022/
18/01/23 06:23:59 INFO mapreduce.Job: Running job: job_1516676845530_0022
18/01/23 06:24:04 INFO mapreduce.Job: Job job_1516676845530_0022 running in uber mode : false
18/01/23 06:24:04 INFO mapreduce.Job:  map 0% reduce 0%
18/01/23 06:24:21 INFO mapreduce.Job:  map 10% reduce 0%
18/01/23 06

## test

In [14]:
%%writefile debug_data.txt
1	5ffgfg truong Minh Xua.n ThAo 5 T4an Ph$uong POtter MidU P4hUong P9Huong P4huOng
2	Xuan Thao xuan thao XuAn Thao T5an P4huong T5An P4huonG

Overwriting debug_data.txt


In [17]:
cat debug_data.txt | python2 ./mapper.py | sort

midu	1	not_ok
minh	1	ok
p4huong	1	not_ok
p4huong	1	not_ok
p4huong	1	not_ok
p4huong	1	ok
p9huong	1	not_ok
ph$uong	1	ok
potter	1	not_ok
t4an	1	ok
t5an	1	not_ok
t5an	1	ok
thao	1	not_ok
thao	1	not_ok
thao	1	ok
thao	1	ok
truong	1	not_ok
xuan	1	not_ok
xuan	1	not_ok
xua.n	1	ok
xuan	1	ok


In [18]:
# cat debug_data.txt | python2 mapper.py | sort | python2 reducer.py

minh	1
ph$uong	1
t4an	1
xua.n	1
xuan	1
